In [46]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import requests
from pandas.io.json import json_normalize
import json
import urllib.request

from difflib import SequenceMatcher

In [17]:
with open('station_information.json', 'r') as json_file:
    json_load = json.load(json_file)

    station_info = pd.DataFrame(json_load['data']['stations'])

In [18]:
station_info.shape

(680, 16)

In [19]:
station_info.head()

,lon,external_id,rental_uris,capacity,station_id,rental_methods,name,eightd_station_services,short_name,eightd_has_key_dispenser,electric_bike_surcharge_waiver,lat,legacy_id,region_id,station_type,has_kiosk
0,-77.053230,082469cc-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",15,1,"[KEY, CREDITCARD]",Eads St & 15th St S,[],31000,False,False,38.858971,1,41,classic,True
1,-77.049232,08246c35-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",17,3,"[KEY, CREDITCARD]",Crystal Dr & 20th St S,[],31002,False,False,38.856425,3,41,classic,True
2,-77.049417,08246cd5-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",16,4,"[KEY, CREDITCARD]",Crystal Dr & 15th St S,[],31003,False,False,38.861056,4,41,classic,True
3,-77.059490,08246d68-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",12,5,"[KEY, CREDITCARD]",Aurora Hills Cmty Ctr / 18th St & S Hayes St,[],31004,False,False,38.857866,5,41,classic,True
4,-77.059936,08246df5-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",19,6,"[KEY, CREDITCARD]",Pentagon City Metro / 12th St & S Hayes St,[],31005,False,False,38.862303,6,41,classic,True


In [20]:
station_loc = station_info[['name','lat','lon','region_id']]

In [21]:
station_loc.head()

,name,lat,lon,region_id
0,Eads St & 15th St S,38.858971,-77.053230,41
1,Crystal Dr & 20th St S,38.856425,-77.049232,41
2,Crystal Dr & 15th St S,38.861056,-77.049417,41
3,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490,41
4,Pentagon City Metro / 12th St & S Hayes St,38.862303,-77.059936,41


In [22]:
regions = [
{
"name": "Alexandria, VA",
"region_id": "40"
},

{
"name": "Arlington, VA",
"region_id": "41"
},
{
"name": "Washington, DC",
"region_id": "42"
},
{
"name": "Montgomery County, MD (North)",
"region_id": "43"
},
{
"name": "Montgomery County, MD (South)",
"region_id": "44"
},
{
"name": "Test & Operations",
"region_id": "48"
},
{
"name": "Fairfax, VA",
"region_id": "104"
},
{
"name": "8D",
"region_id": "128"
},
{
"name": "Prince George's County",
"region_id": "133"
},
{
"name": "Falls Church, VA",
"region_id": "152"
}
]


regions_dict = {
40: "Alexandria, VA",
41: "Arlington, VA", 
42: "Washington, DC",
43: "Montgomery County, MD (North)",
44: "Montgomery County, MD (South)",
48: "Test & Operations",
104: "Fairfax, VA",
128: "8D",
133: "Prince George's County",
152: "Falls Church, VA"
}

In [24]:
station_loc['region_id'] = station_loc['region_id'].fillna(0)
station_loc['region_id'] = station_loc['region_id'].astype('int64')
station_loc['region'] = station_loc['region_id'].map(regions_dict)

/var/folders/68/lmdx_4hd4msbstk8y2t5phyh0000gn/T/ipykernel_39107/1294926311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_loc['region_id'] = station_loc['region_id'].fillna(0)
/var/folders/68/lmdx_4hd4msbstk8y2t5phyh0000gn/T/ipykernel_39107/1294926311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_loc['region_id'] = station_loc['region_id'].astype('int64')
/var/folders/68/lmdx_4hd4msbstk8y2t5phyh0000gn/T/ipykernel_39107/1294926311.py:3: SettingWithCopyWarning: 
A value is trying t

In [28]:
station_loc.head()

,name,lat,lon,region_id,region
0,Eads St & 15th St S,38.858971,-77.053230,41,"Arlington, VA"
1,Crystal Dr & 20th St S,38.856425,-77.049232,41,"Arlington, VA"
2,Crystal Dr & 15th St S,38.861056,-77.049417,41,"Arlington, VA"
3,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490,41,"Arlington, VA"
4,Pentagon City Metro / 12th St & S Hayes St,38.862303,-77.059936,41,"Arlington, VA"


In [29]:
station_loc['name'].nunique()

680

In [30]:
df_2011 = pd.read_csv('cleaned_data/df_2011.csv', parse_dates = ['start_date', 'end_date'])

In [31]:
df_2011

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual
...,...,...,...,...,...,...,...,...,...
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,15th & P St NW,31214,17th & Corcoran St NW,W01459,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,Convention Center / 7th & M St NW,31201,15th & P St NW,W01262,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,Lamont & Mt Pleasant NW,31602,Park Rd & Holmead Pl NW,W00998,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,21st & I St NW,31222,New York Ave & 15th St NW,W00042,Member


Check that set of all start stations is the same as set of all end stations

In [32]:
start_stations_2011 = df_2011['start_station'].unique()
end_stations_2011 = df_2011['end_station'].unique()

In [33]:
len(start_stations_2011), len (end_stations_2011)

(144, 144)

In [231]:
len (np.intersect1d(start_stations_2011, end_stations_2011))

144

In [34]:
stations_2011 = pd.DataFrame(start_stations_2011)

In [35]:
stations_2011 = stations_2011.rename(columns = {0:'station'})

In [36]:
stations_2011

,station
0,5th & F St NW
1,14th & Harvard St NW
2,Georgia & New Hampshire Ave NW
3,10th & U St NW
4,Adams Mill & Columbia Rd NW
...,...
139,3000 Connecticut Ave NW / National Zoo
140,Benning Rd & East Capitol St NE / Benning Rd M...
141,Anacostia Ave & Benning Rd NE / River Terrace
142,15th St & Massachusetts Ave SE


In [37]:
station_loc

,name,lat,lon,region_id,region
0,Eads St & 15th St S,38.858971,-77.053230,41,"Arlington, VA"
1,Crystal Dr & 20th St S,38.856425,-77.049232,41,"Arlington, VA"
2,Crystal Dr & 15th St S,38.861056,-77.049417,41,"Arlington, VA"
3,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490,41,"Arlington, VA"
4,Pentagon City Metro / 12th St & S Hayes St,38.862303,-77.059936,41,"Arlington, VA"
...,...,...,...,...,...
675,16th & Harvard St NW,38.926102,-77.036650,42,"Washington, DC"
676,First St & Fayette St,38.817019,-77.050105,40,"Alexandria, VA"
677,Penrose Sq/Columbia Pike & S Barton St,38.863508,-77.084259,41,"Arlington, VA"
678,S Wakefield St & 28th Rd S,38.844040,-77.098093,41,"Arlington, VA"


In [38]:
stations_2011['region'] = np.zeros(stations_2011.shape[0])
stations_2011['lat'] = np.zeros(stations_2011.shape[0])
stations_2011['lon'] = np.zeros(stations_2011.shape[0])

In [39]:
stations_2011.head()

,station,region,lat,lon
0,5th & F St NW,0.0,0.0,0.0
1,14th & Harvard St NW,0.0,0.0,0.0
2,Georgia & New Hampshire Ave NW,0.0,0.0,0.0
3,10th & U St NW,0.0,0.0,0.0
4,Adams Mill & Columbia Rd NW,0.0,0.0,0.0


In [40]:
station_loc.head()

,name,lat,lon,region_id,region
0,Eads St & 15th St S,38.858971,-77.053230,41,"Arlington, VA"
1,Crystal Dr & 20th St S,38.856425,-77.049232,41,"Arlington, VA"
2,Crystal Dr & 15th St S,38.861056,-77.049417,41,"Arlington, VA"
3,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490,41,"Arlington, VA"
4,Pentagon City Metro / 12th St & S Hayes St,38.862303,-77.059936,41,"Arlington, VA"


In [47]:
station_loc.to_csv('station_loc.csv')

In [41]:
for i in stations_2011.index:
    for j in station_loc.index:
        if stations_2011.loc[i,'station'] == station_loc.loc[j, 'name']:
            stations_2011.loc[i, 'lat'] = station_loc.loc[j, 'lat']
            stations_2011.loc[i, 'lon'] = station_loc.loc[j, 'lon']
            stations_2011.loc[i, 'region'] = station_loc.loc[j, 'region']
       


In [45]:
stations_2011.head(30)

,station,region,lat,lon
0,5th & F St NW,"Washington, DC",38.897222,-77.019347
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200
2,Georgia & New Hampshire Ave NW,"Washington, DC",38.936684,-77.024181
3,10th & U St NW,"Washington, DC",38.917200,-77.025900
4,Adams Mill & Columbia Rd NW,"Washington, DC",38.922925,-77.042581
5,3rd & D St SE,"Washington, DC",38.885100,-77.002300
6,14th & Rhode Island Ave NW,"Washington, DC",38.908600,-77.032300
7,15th & P St NW,"Washington, DC",38.909801,-77.034427
8,Van Ness Metro / UDC,"Washington, DC",38.944551,-77.063896
9,Crystal City Metro / 18th & Bell St,0.0,0.000000,0.000000


Create a list of the missing station names (i.e. stations for which 'lat' and 'lon' are not available)

In [241]:
missing_stations = []
for i in stations_2011.index:
    if stations_2011.loc[i,'lat'] == 0 or stations_2011.loc[i,'lat'] == 0:
        missing_stations.append(stations_2011.loc[i,'station'])

In [242]:
missing_stations

['Crystal City Metro / 18th & Bell St',
 '21st & M St NW',
 'Eastern Market Metro / Pennsylvania Ave & 7th St SE',
 'Connecticut Ave & Newark St NW / Cleveland Park',
 '18th & Eads St.',
 '19th & L St NW',
 '23rd & Crystal Dr',
 'Aurora Hills Community Ctr/18th & Hayes St',
 'S Joyce & Army Navy Dr',
 'Georgia Ave and Fairmont St NW',
 '20th & Crystal Dr',
 'S Glebe & Potomac Ave',
 'USDA / 12th & Independence Ave SW',
 '27th & Crystal Dr',
 'Pentagon City Metro / 12th & S Hayes St',
 '12th & Army Navy Dr',
 '26th & S Clark St',
 '15th & Crystal Dr',
 'Eads & 22nd St S',
 '1st & N St  SE',
 'Lynn & 19th St North',
 'N Rhodes & 16th St N',
 'Rosslyn Metro / Wilson Blvd & Ft Myer Dr',
 'Wilson Blvd & Franklin Rd',
 '11th & H St NE']

I went and found the "lon" and "lat" for 23 of the 25 stations in "missing". <br>
I did this using geopandas in a separate notebook, and saved the dataframe as a csv file. <br>



In [50]:
#Import missing_2011

missing_2011 = pd.read_csv('cleaned_data/missing_2011.csv')



In [51]:
missing_2011

,station,region,lat,lon
0,Crystal City Metro / 18th & Bell St,"Arlington, VA",38.857756,-77.051196
1,21st & M St NW,"Washington, DC",38.905107,-77.057402
2,Eastern Market Metro / Pennsylvania Ave & 7th ...,"Washington, DC",38.884056,-76.995262
3,Connecticut Ave & Newark St NW / Cleveland Park,0,38.934267,-77.057979
4,18th & Eads St.,0,0.000000,0.000000
5,19th & L St NW,"Washington, DC",38.903799,-77.053958
6,23rd & Crystal Dr,0,38.853166,-77.050493
7,Aurora Hills Community Ctr/18th & Hayes St,0,38.857792,-77.059103
8,S Joyce & Army Navy Dr,0,38.865710,-77.061773
9,Georgia Ave and Fairmont St NW,"Washington, DC",38.924900,-77.022200


In [57]:
stations_2011.head()

,station,region,lat,lon
0,5th & F St NW,"Washington, DC",38.897222,-77.019347
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200
2,Georgia & New Hampshire Ave NW,"Washington, DC",38.936684,-77.024181
3,10th & U St NW,"Washington, DC",38.917200,-77.025900
4,Adams Mill & Columbia Rd NW,"Washington, DC",38.922925,-77.042581


In [58]:
# Now fill missing "lat" and "lon" values in "stations_2011" with the respective values from "missing_2011"

for i in range(len(stations_2011)):
    if stations_2011.loc[i, 'lat'] == 0:
        for j in range (len(missing_2011)):
            if stations_2011.loc[i, 'station'] == missing_2011.loc[j, 'station']:
                stations_2011.loc[i, 'lat'] = missing_2011.loc[j, 'lat']
                stations_2011.loc[i, 'lon'] = missing_2011.loc[j, 'lon']


In [63]:
# Check that there are only 2 missing data points
len(stations_2011[stations_2011['lat'] == 0])

2

In [64]:
# Now I'll just have to fill the two missing rows manually using googlemaps. 

stations_2011[stations_2011['lat'] == 0]

,station,region,lat,lon
27,18th & Eads St.,0.0,0.0,0.0
88,15th & Crystal Dr,0.0,0.0,0.0


In [65]:
# 18th & Eads St. : 38.85757, -77.05307 (Arlington, VA)
# 15th & Crystal Dr: 38.86032, -77.04934 (Arlington, VA)

stations_2011.loc[27, 'region'] = 'Arlington, VA'
stations_2011.loc[27, 'lat'] = 38.85757
stations_2011.loc[27, 'lon'] = -77.05307

stations_2011.loc[88, 'region'] = 'Arlington, VA'
stations_2011.loc[88, 'lat'] = 38.86032
stations_2011.loc[88, 'lon'] = -77.04934


In [66]:
stations_2011

,station,region,lat,lon
0,5th & F St NW,"Washington, DC",38.897222,-77.019347
1,14th & Harvard St NW,"Washington, DC",38.926800,-77.032200
2,Georgia & New Hampshire Ave NW,"Washington, DC",38.936684,-77.024181
3,10th & U St NW,"Washington, DC",38.917200,-77.025900
4,Adams Mill & Columbia Rd NW,"Washington, DC",38.922925,-77.042581
...,...,...,...,...
139,3000 Connecticut Ave NW / National Zoo,"Washington, DC",38.930282,-77.055599
140,Benning Rd & East Capitol St NE / Benning Rd M...,"Washington, DC",38.889935,-76.937230
141,Anacostia Ave & Benning Rd NE / River Terrace,"Washington, DC",38.896544,-76.960120
142,15th St & Massachusetts Ave SE,"Washington, DC",38.887320,-76.983569


In [69]:
stations_2011.to_csv('cleaned_data/stations_2011.csv', index = False)